In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import numpy as np


In [ ]:
#get post-proc dataset
data=pd.read_csv('../data/data_2024.csv')
#retrive countries geometries, locations ids (iso-alpha codes) ---> original data source from kaggle
# Opening JSON file
f = open('../data/world-countries.json')
# returns JSON object as
# a dictionary
geoj = json.load(f)

In [ ]:
#choropleth map - world level. color by best estimate of total deaths per country. Aggregate data
#to shrink color scale, log of the best estimate has been used.
#to make the scale easy to read, the legend has beed re-mapped to the non-log data.
data_agg=data.groupby(by=['country','iso_alpha'])[['best','deaths_civilians']].sum().reset_index()
data_agg['best_log']=np.log10(data_agg.best+1)
data_agg['deaths_civilians_log']=np.log10(data_agg.deaths_civilians+1)
#map code
fig = px.choropleth_mapbox(data_agg,  locations='iso_alpha' ,color='best_log', geojson=geoj,
                           color_continuous_scale="Viridis",mapbox_style="open-street-map",
                           zoom=1,
                           opacity=0.5,
                           hover_name='country',
                           hover_data=['deaths_civilians','best'])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},coloraxis_colorbar=dict(
        title="Deaths",
        tickvals=[0, 1, 2, 3, 4, 5],
        ticktext=["1", "10", "100", "1k", "10k", "100k"],
    ),)

fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:
#choropleth map - world level. color by best estimate of total deaths per country. aggregate data on weekly basis
#to shrink color scale, eval "log"
#animated version
#sort data by date - this is to get the correct animation sequency

data_agg_weekly=data.groupby(by=['country','iso_alpha','date_start_wymd','region'])[['best','deaths_civilians']].sum().reset_index()
data_agg_weekly['best_log']=np.log10(data_agg_weekly.best+1)
data_agg_weekly['deaths_civilians_log']=np.log10(data_agg_weekly.deaths_civilians+1)
#sort data by date - this is to get the correct animation sequency
data_agg_weekly["date_start_wymd"]=pd.to_datetime(data_agg_weekly["date_start_wymd"]).dt.strftime('%Y-%m-%d')
data_agg_weekly.sort_values(by=['date_start_wymd'], inplace=True)
data_agg_weekly.reset_index(drop=True, inplace=True)
#focus on spefic region - e.g. Africa
data_agg_weekly=data_agg_weekly[data_agg_weekly.region=='Africa']
#map code
fig = px.choropleth_mapbox(data_agg_weekly,  locations='iso_alpha' ,color='best', geojson=geoj,
                           color_continuous_scale="Viridis",
                           mapbox_style="open-street-map",
                           zoom=1,
                           opacity=0.5,
                           hover_name='country',
                           hover_data=['deaths_civilians','best'] ,animation_frame="date_start_wymd", animation_group="country")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},coloraxis_colorbar=dict(
        title="Deaths",
      
    ),height=800)


fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:

#focus on specific country (to be selected -Angola in this case) -- use scatter map ox to show events location more precisely.
#in this case the color scale refer to deaths_civilians.
#map center point to the selected country
country_sel='Burkina Faso'
data_ff=data[data.country==country_sel]
fig = px.scatter_mapbox(data_ff, lat="latitude", lon="longitude", color='deaths_civilians',
                     hover_name="country", size="deaths_civilians", zoom=4, height=500, 
                     center = {"lat": data_ff.latitude.values[0], "lon":data_ff.longitude.values[0]},
                     color_continuous_scale=px.colors.cyclical.IceFire
                     )

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()